# 실습 순서

STEP1. 아이디어 개요
 
 비슷한 이미지만 골라서 화면에 표시하기.

<이용할 것들>

cv2.calcHist():히스토그램 만들어주는 함수

cv2.compareHist(): 히스토그램끼리의 유사성 계산

sys.argv:이밎 파일 경로 명령줄에서 입력

<pre-code>
    i)프로그램 실행
    
    ii)이미지 파일 부르기
    
    iii)비슷한 이미지 5개 선정 후 표시
    iv)끝
    
    
STEP2. 아이디어 구체화
    
    pre-code의 iii
 -유사도를 통해서 순서매김 -> 순서상 상위 5개 고르기
    
STEP3. 핵심 알고리즘-유사도 계산방법
    
    -히스토그램으로 비교
    -Open CV기능 이용(compareHist()함수)
    
STEP4.기능의 모듈화
    
    build_histogram_db() : 검색대상 이미지를 부르고 
    히스토그램화(검색 대상:CIFAR-100이미지)

    search(): 유사도 계산 및 정렬에 필요

STEP5.설계과정 완성
    
각 함수에 입력되고 반환될ㄹ 데이터의 형태와 변수명지정
    
    1. histogram_db{키 : 이미지 이름, 값: 히스토그램}
    2, 선택된 이미지는 입력 이미지로 선택함
    3, 이때, 변수명은 target_histogram
    4. compareHist()함수를 통해 유사도 계산
    5. 결과: result{키: 이미지 이름, 값: 유사도}
    6. 정렬
    7. 상위 5개 선택 후 result에 남기기
    
STEP6. 코드 구현

In [ ]:

def get_histogram(image):
    histogram = []

    # Create histograms per channels, in 4 bins each.
    for i in range(3):
        channel_histogram = cv2.calcHist(images=[image],
                                         channels=[i],
                                         mask=None,
                                         histSize=[4],  # 히스토그램 구간을 4개로 한다.
                                         ranges=[0, 256])
        histogram.append(channel_histogram)  

    histogram = np.concatenate(histogram)
    histogram = cv2.normalize(histogram, histogram)

    return histogram

In [ ]:
# get_histogram() 확인용 코드
filename = train[b'filenames'][0].decode()
file_path = os.path.join(images_dir_path, filename)
image = cv2.imread(file_path)
histogram = get_histogram(image)
histogram

In [ ]:
def build_histogram_db():
    histogram_db = {}

    #디렉토리에 모아 둔 이미지 파일들을 전부 리스트업합니다. 
    path = images_dir_path
    file_list = os.listdir(images_dir_path)

    for file_name in tqdm(file_list):
        file_path = os.path.join(images_dir_path, file_name)
        image = cv2.imread(file_path)

        histogram = get_histogram(image)

        histogram_db[file_name] = histogram

    return histogram_db


In [ ]:
histogram_db = build_histogram_db()
histogram_db['adriatic_s_001807.png']

In [ ]:
def get_target_histogram():
    filename = input("이미지 파일명을 입력하세요: ")
    if filename not in histogram_db:
        print('유효하지 않은 이미지 파일명입니다.')
        return None
    return histogram_db[filename]

In [ ]:
target_histogram = get_target_histogram()
target_histogram

In [ ]:
def search(histogram_db, target_histogram, top_k=5):
    results = {}

    # Calculate similarity distance by comparing histograms.
    for file_name, histogram in tqdm(histogram_db.items()):
        distance = cv2.compareHist(H1=target_histogram,
                                   H2=histogram,
                                   method=cv2.HISTCMP_CHISQR)

        results[file_name] = distance

    results = dict(sorted(results.items(), key=lambda item: item[1])[:top_k])

    return results


In [ ]:
result = search(histogram_db, target_histogram)
result

In [ ]:
def show_result(result):
    f=plt.figure(figsize=(10,3))
    for idx, filename in enumerate(result.keys()):    
        img_path = os.path.join(images_dir_path, filename)
        im = f.add_subplot(1,len(result),idx+1)
        img = Image.open(img_path)
        im.imshow(img)


In [ ]:
show_result(result)

In [ ]:
target_histogram = get_target_histogram()
result = search(histogram_db, target_histogram)
show_result(result)